In [1]:
print('hello, world!')

hello, world!


In [17]:
import os
import datasets

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    T5TokenizerFast,
    get_linear_schedule_with_warmup
)

In [18]:
k_model = "mrm8488/t5-base-finetuned-wikiSQL" 
model = T5ForConditionalGeneration.from_pretrained(k_model)
tokenizer = T5Tokenizer.from_pretrained(k_model)
ftokenizer = T5TokenizerFast.from_pretrained(k_model)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [ ]:
class SynthData:

    def __init__(self, dataset='semiotic/spider_dataset_tuning'): 
        self.data = datasets.load_dataset(dataset)
        #self.data = datasets.load_from_disk('data/spider_tuning_dict')

    def format_request(self, question, schema): 
        input_template = "Schema: {schema} translate English to SQL: {question} </s>"
        return input_template.format(schema=schema, question=question)
    
    def get_subset(self, subset='train', type='synthetic_joint'): 
        return self.data[subset].filter(lambda x: x['type'] == type)
    
    def prepare_sources(self, subset='train', type='synthetic_joint', data_dir='data'):
        # Ensure data_dir exists
        os.makedirs(data_dir, exist_ok=True)

        # Create or clear existing files
        open(f'{data_dir}/{subset}.source', 'w').close()
        open(f'{data_dir}/{subset}.target', 'w').close()

        # Get subset data
        subset_data = self.get_subset(subset=subset, type=type)

        # Append data to the files
        for i, row in enumerate(subset_data):
            with open(f'{data_dir}/{subset}.source', 'a') as f_source, \
                open(f'{data_dir}/{subset}.target', 'a') as f_target:
                f_source.write(self.format_request(row['question'], row['schema']) + '\n')
                f_target.write(row['query'] + ' </s>\n')

    def get_unk_tokens(self, tokenizer, value): 
        unk_tokens = []
        tokens = set(tokenizer.tokenize(value))
        for token in tokens: 
            if tokenizer.convert_tokens_to_ids(token) == tokenizer.unk_token_id: 
                unk_tokens.append(token)
        return unk_tokens

    def get_dataset_unk_tokens(self, tokenizer, dataset): 
        unk_tokens = []
        for i, row in enumerate(dataset): 
            text = self.format_request(row['question'], row['schema']) + ' ' + row['query']
            unks = self.get_unk_tokens(tokenizer, text)
            unk_tokens.extend(unks)
        return list(set(unk_tokens))

    def get_all_unk_tokens(self, tokenizer): 
        unk_tokens = []
        unk_tokens.extend(self.get_dataset_unk_tokens(tokenizer, self.data['train']))
        unk_tokens.extend(self.get_dataset_unk_tokens(tokenizer, self.data['val']))
        unk_tokens.extend(self.get_dataset_unk_tokens(tokenizer, self.data['test']))
        return list(set(unk_tokens))

sd = SynthData()

In [89]:
sd.get_all_unk_tokens(tokenizer)

Schema: ['CREATE TABLE course (course_id NUMBER, staring_date TEXT, course TEXT);', 'CREATE TABLE teacher (teacher_id NUMBER, name TEXT, age TEXT, hometown TEXT);', 'CREATE TABLE course_arrange (course_id NUMBER, teacher_id NUMBER, grade NUMBER, FOREIGN KEY (teacher_id) REFERENCES teacher(teacher_id), FOREIGN KEY (course_id) REFERENCES course(course_id));'] translate English to SQL: List the name of teachers whose hometown is not `` Little Lever Urban District '' . </s> select name from teacher where hometown != "little lever urban district"
Schema: ['CREATE TABLE course (course_id NUMBER, staring_date TEXT, course TEXT);', 'CREATE TABLE teacher (teacher_id NUMBER, name TEXT, age TEXT, hometown TEXT);', 'CREATE TABLE course_arrange (course_id NUMBER, teacher_id NUMBER, grade NUMBER, FOREIGN KEY (teacher_id) REFERENCES teacher(teacher_id), FOREIGN KEY (course_id) REFERENCES course(course_id));'] translate English to SQL: What are the names of the teachers whose hometown is not `` Little

['``', '<']

In [7]:
train = sd.get_subset(subset="train")
val = sd.get_subset(subset="val")
test = sd.get_subset(subset="test")

Filter: 100%|██████████| 870/870 [00:00<00:00, 79249.53 examples/s]


In [8]:
train

Dataset({
    features: ['type', 'question', 'query', 'db_id', 'schema'],
    num_rows: 906
})

In [33]:
tokenizer.unk_token_id

2

In [46]:
q = "SELECT count(*) FROM Movie WHERE YEAR  <  2000 </s>"
tokens = set(tokenizer.tokenize(q))
for token in tokens: 
    if tokenizer.convert_tokens_to_ids(token) == tokenizer.unk_token_id: 
        print(token)

<


In [82]:
def get_unk_tokens(sd, tokenizer, value): 
    unk_tokens = []
    tokens = set(tokenizer.tokenize(value))
    for token in tokens: 
        if tokenizer.convert_tokens_to_ids(token) == tokenizer.unk_token_id: 
            unk_tokens.append(token)
    return unk_tokens

def get_dataset_unk_tokens(sd, dataset, tokenizer): 
    unk_tokens = []
    for i, row in enumerate(dataset): 
        text = sd.format_request(row['question'], row['schema']) + ' ' + row['query']
        unks = get_unk_tokens(sd, tokenizer, text)
        unk_tokens.extend(unks)
    return list(set(unk_tokens))
        

t = get_dataset_unk_tokens(sd, val, tokenizer)
t

['<']

In [78]:
get_unk_tokens(sd, tokenizer, '~`!@#$%^&*()_+<>')

['^', '~`', '<']

In [76]:
tokenizer.convert_tokens_to_ids()

2